In [22]:
import polygraphs as pg
import torch

def generator(epsilon, trials, size):
    # Action B yields Bernoulli payoff of 1 (success)
    # with probability p (= 0.5 + e)
    probs = pg.init.halfs(size) + epsilon
    
    # Number of Bernoulli trials
    count = pg.init.zeros(size) + trials

    # Each node gets a private signal that provides information
    # about whether action B is indeed a good action
    sampler = torch.distributions.binomial.Binomial(total_count=count,
                                                    probs=probs)
    return sampler

# Set random seed
pg.random(123)
epsilon = 0.1
trials = 10
size = (1,)

g = generator(epsilon, trials, size)

# Pick a prior, any value between 0 and 1
params = pg.hyperparameters.InitHyperParameters()
params.constant.value = 0.5
prior = pg.init.constant(size, params)
print("Prior is", prior)

# Get two samples
s1 = g.sample()
s2 = g.sample()
print("1st sample is", s1)
print("2nd sample is", s2)

# At time step 1, update prior using first sample
e1 = pg.ops.math.Evidence(g.logits, s1, g.total_count)

posterior = pg.ops.math.bayes(prior, e1)
print("Posterior( t=1) is", posterior)

# At time step 2, update prior (posterior of previous step) using second sample
e2 = pg.ops.math.Evidence(g.logits, s2, g.total_count)

posterior = pg.ops.math.bayes(posterior, e2)
print("Posterior (t=2) is", posterior)

# At time 1, update prior using both samples
e3 = pg.ops.math.Evidence(g.logits, s1 + s2, 2 * g.total_count)

posterior = pg.ops.math.bayes(prior, e3)
print("Posterior is", posterior)


Prior is tensor([0.5000])
1st sample is tensor([9.])
2nd sample is tensor([7.])
Posterior( t=1) is tensor([0.9624])
Posterior (t=2) is tensor([0.9924])
Posterior is tensor([0.9924])


In [30]:
# Set random seed
pg.random(123)
epsilon = 0.1
trials = 10
size = (1,)
certainty = 0.6

g = generator(epsilon, trials, size)

# Pick a prior, any value between 0 and 1
params = pg.hyperparameters.InitHyperParameters()
params.constant.value = 0.5
prior = pg.init.constant(size, params)
print("Prior is", prior)

# Get two samples
s1 = g.sample()
s2 = g.sample()
print("1st sample is", s1)
print("2nd sample is", s2)

# At time step 1, update prior using first sample
e1 = pg.ops.math.Evidence(g.logits, s1, g.total_count)

posterior = pg.ops.math.jeffrey(prior, e1, certainty)
print("Posterior( t=1) is", posterior)

# At time step 2, update prior (posterior of previous step) using second sample
e2 = pg.ops.math.Evidence(g.logits, s2, g.total_count)

posterior = pg.ops.math.jeffrey(posterior, e2, certainty)
print("Posterior (t=2) is", posterior)

# At time 1, update prior using both samples
e3 = pg.ops.math.Evidence(g.logits, s1 + s2, 2 * g.total_count)

posterior = pg.ops.math.jeffrey(prior, e3, certainty)
print("Posterior is", posterior)


Prior is tensor([0.5000])
1st sample is tensor([9.])
2nd sample is tensor([7.])
Posterior( t=1) is tensor([0.7735])
Posterior (t=2) is tensor([0.8619])
Posterior is tensor([0.7919])
